In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense
import shap
import matplotlib.pyplot as plt

# Generate random data
np.random.seed(42)
num_samples = 1000
temperature = np.random.rand(num_samples) * 40  # Temperature in Celsius
humidity = np.random.rand(num_samples) * 100  # Humidity in percentage
pressure = np.random.rand(num_samples) * 20 + 980  # Pressure in hPa
wind_speed = np.random.rand(num_samples) * 15  # Wind speed in m/s

# Create a DataFrame
data = pd.DataFrame({
    'temperature': temperature,
    'humidity': humidity,
    'pressure': pressure,
    'wind_speed': wind_speed
})

# Introduce padding
data['temperature'].iloc[950:] = 0
data['humidity'].iloc[950:] = 0
data['pressure'].iloc[950:] = 0

# Split into features and target
X = data[['temperature', 'humidity', 'pressure']].values
y = data['wind_speed'].values

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape X to be 3D for LSTM (samples, timesteps, features)
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Masking(mask_value=0.0, input_shape=(1, X_train.shape[2])))  # Masking layer
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=2)

# Create a SHAP explainer for the LSTM model
explainer = shap.KernelExplainer(model, X_train)
shap_values = explainer.shap_values(X_test)

# Since we are predicting a single output, we use shap_values[0]
# SHAP summary plot
shap.summary_plot(shap_values[0], X_test, feature_names=['temperature', 'humidity', 'pressure'])

Epoch 1/10


/var/folders/01/5ryz4pnn581dj9gk6r1nn5qr0000gn/T/ipykernel_23782/1367394781.py:27: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['temperature'].iloc[950:] = 0
/var/folders/01/5ryz4pnn581dj9gk6r1nn5qr0000gn/T/ipykernel_23782/1367394781.p

20/20 - 1s - 71ms/step - loss: 69.7318 - val_loss: 75.7387
Epoch 2/10
20/20 - 0s - 16ms/step - loss: 68.5255 - val_loss: 74.4312
Epoch 3/10
20/20 - 0s - 15ms/step - loss: 67.2431 - val_loss: 72.9512
Epoch 4/10
20/20 - 0s - 15ms/step - loss: 65.7398 - val_loss: 71.2187
Epoch 5/10
20/20 - 0s - 15ms/step - loss: 63.9191 - val_loss: 69.0960
Epoch 6/10
20/20 - 0s - 15ms/step - loss: 61.7117 - val_loss: 66.4935
Epoch 7/10
20/20 - 0s - 15ms/step - loss: 59.0776 - val_loss: 63.3985
Epoch 8/10
20/20 - 0s - 15ms/step - loss: 55.9986 - val_loss: 59.8827
Epoch 9/10
20/20 - 0s - 16ms/step - loss: 52.5743 - val_loss: 55.8989
Epoch 10/10
20/20 - 0s - 15ms/step - loss: 48.7868 - val_loss: 51.6246


Using 800 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


DimensionError: Instance must have 1 or 2 dimensions!

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Masking, TimeDistributed

# Sample data (dummy data for illustration)
# Assume 10 samples, each with 12 hours of historical data (12 time steps), and 3 features
historical_data = np.random.rand(10, 12, 3).astype(np.float32)
# Corresponding future wind speeds for the next 11 hours
future_wind_speeds = np.random.rand(10, 11, 1).astype(np.float32)

# Define the model
input_seq = Input(shape=(12, 3))
masked_input = Masking(mask_value=-1)(input_seq)
encoder = LSTM(64, return_state=True)
encoder_outputs, state_h, state_c = encoder(masked_input)
encoder_states = [state_h, state_c]

# Set up the decoder
decoder_input = Input(shape=(11, 1))  # Shifted right future wind speed
masked_decoder_input = Masking(mask_value=-1)(decoder_input)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(masked_decoder_input, initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(1))  # Output layer
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([input_seq, decoder_input], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
# Shift future_wind_speeds to the right by one time step as decoder input
decoder_input_data = np.zeros_like(future_wind_speeds)
decoder_input_data[:, 1:, :] = future_wind_speeds[:, :-1, :]
decoder_input_data[:, 0, :] = 0  # Assume start of sequence is 0 or another appropriate value

model.fit([historical_data, decoder_input_data], future_wind_speeds, epochs=20, batch_size=1)

# Evaluate and predict
predicted_wind_speeds = model.predict([historical_data, decoder_input_data])

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 0.3350
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1500
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1364
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1145
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1004
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0894
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1052
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0886
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0984
Epoch 10/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0767
Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0876
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0866
Epoch 13/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0868
Epoch 14/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0958
Epoch 15/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0894
Epoc